# Variational AutoEncoder Generator Tensorflow
This should open a pre-trained VAE and become a Data generator

#### References:
* http://kvfrans.com/variational-autoencoders-explained/
* https://github.com/kvfrans/variational-autoencoder
* https://github.com/int8/VAE_tensorflow
* http://int8.io/variational-autoencoder-in-tensorflow/
* http://blog.fastforwardlabs.com/2016/08/22/under-the-hood-of-the-variational-autoencoder-in.html
* http://blog.fastforwardlabs.com/2016/08/12/introducing-variational-autoencoders-in-prose-and.html
* https://blog.keras.io/building-autoencoders-in-keras.html
* https://jaan.io/what-is-variational-autoencoder-vae-tutorial/
* https://arxiv.org/pdf/1606.05908.pdf
* https://arxiv.org/pdf/1312.6114.pdf
* http://wiseodd.github.io/techblog/2016/12/10/variational-autoencoder/
* https://www.tensorflow.org/get_started/embedding_viz
* https://www.youtube.com/watch?v=eBbEDRsCmv4
* https://www.youtube.com/watch?v=bbOFvxbMIV0
* https://www.youtube.com/watch?v=P78QYjWh5sM
* https://github.com/normanheckscher/mnist-tensorboard-embeddings
* http://projector.tensorflow.org/

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from pylab import *
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = str(0)

In [2]:
from models import VAE_Segnet_Generator
model = VAE_Segnet_Generator(latent_size = 40)
model_in = model.input
model_out = model.output

In [24]:
# Select only the convs
list_params = [v for v in tf.global_variables() if "z_matrix" in v.name or "conv" in v.name or "Batch" in v.name]
#list_params = [v for v in tf.global_variables() if "z_matrix" in v.name or "conv" in v.name]

In [25]:
# Problem inside the model the Batchnorm variables are BatchNorm_[1...7] but on the file 
# they are BatchNorm_7...13
list_params

[<tf.Variable 'LATENT/z_matrix/weights:0' shape=(40, 2592) dtype=float32_ref>,
 <tf.Variable 'LATENT/z_matrix/bias:0' shape=(2592,) dtype=float32_ref>,
 <tf.Variable 'DECODER/dconv1/weights:0' shape=(3, 3, 32, 32) dtype=float32_ref>,
 <tf.Variable 'DECODER/dconv1/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'BatchNorm/beta:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'BatchNorm/gamma:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'BatchNorm/moving_mean:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'BatchNorm/moving_variance:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'DECODER/dconv2/weights:0' shape=(3, 3, 32, 32) dtype=float32_ref>,
 <tf.Variable 'DECODER/dconv2/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'BatchNorm_1/beta:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'BatchNorm_1/gamma:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'BatchNorm_1/moving_mean:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'BatchNorm_1/moving_variance:0' shape=(32,

In [23]:
tf.global_variables()

[<tf.Variable 'LATENT/z_matrix/weights:0' shape=(40, 2592) dtype=float32_ref>,
 <tf.Variable 'LATENT/z_matrix/bias:0' shape=(2592,) dtype=float32_ref>,
 <tf.Variable 'DECODER/dconv1/weights:0' shape=(3, 3, 32, 32) dtype=float32_ref>,
 <tf.Variable 'DECODER/dconv1/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'BatchNorm/beta:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'BatchNorm/gamma:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'BatchNorm/moving_mean:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'BatchNorm/moving_variance:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'DECODER/dconv2/weights:0' shape=(3, 3, 32, 32) dtype=float32_ref>,
 <tf.Variable 'DECODER/dconv2/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'BatchNorm_1/beta:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'BatchNorm_1/gamma:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'BatchNorm_1/moving_mean:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'BatchNorm_1/moving_variance:0' shape=(32,

### Define saver objects to load the VAE generator variables

In [14]:
# Define the saver object to load only the conv variables
saver_load_autoencoder = tf.train.Saver(var_list=list_params)

### Build Graph and create session

In [17]:
init = tf.global_variables_initializer()

# Avoid allocating the whole memory
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.1)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

#sess = tf.Session()
sess.run(init)

# Initialize all random variables (Weights/Bias)
sess.run(tf.global_variables_initializer())

# Restore only the weights (From AutoEncoder)
saver_load_autoencoder.restore(sess, "/home/leoara01/work/Variational_AutoEncoder/src/notebooks/tensorflow/save/model-309")

INFO:tensorflow:Restoring parameters from /home/leoara01/work/Variational_AutoEncoder/src/notebooks/tensorflow/save/model-309


InvalidArgumentError: Assign requires shapes of both tensors to match. lhs shape= [32] rhs shape= [64]
	 [[Node: save_2/Assign = Assign[T=DT_FLOAT, _class=["loc:@BatchNorm/beta"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/gpu:0"](BatchNorm/beta, save_2/RestoreV2/_79)]]

Caused by op 'save_2/Assign', defined at:
  File "/home/leoara01/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/leoara01/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/leoara01/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/leoara01/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/leoara01/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File

### Generate some input from the latent space

In [ ]:
# Create a latent as a normal distribution with size 20, mean 0, variance 1
latent = np.random.normal(0, 1, [1,20])

#latent = np.ones([1,20])
#latent[0,1:10] = -1

out_img = sess.run(model_out, feed_dict={model_in:latent})
plt.imshow(out_img.reshape([28,28]))
plt.show()

### Generate a video

In [ ]:
dpi = 50

def ani_frame():
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_aspect('equal')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    im = ax.imshow(np.random.rand(28,28),cmap='gray',interpolation='nearest')
    #latent = np.random.normal(0, 1, [1,20])
    #out_img = sess.run(model_out, feed_dict={model_in:latent})
    #im = ax.imshow(out_img.reshape([28,28]),cmap='gray',interpolation='nearest')
    
    im.set_clim([0,1])
    fig.set_size_inches([5,5])


    tight_layout()


    def update_img(n):
        #tmp = rand(300,300)
        #im.set_data(tmp)
        latent = np.random.normal(0, 1, [1,20])
        out_img = sess.run(model_out, feed_dict={model_in:latent})
        im.set_data(out_img.reshape([28,28]))
        return im

    #legend(loc=0)
    ani = animation.FuncAnimation(fig,update_img,300,interval=30)
    writer = animation.writers['ffmpeg'](fps=30)

    ani.save('demo.mp4',writer=writer,dpi=dpi)
    return ani

In [ ]:
ani_frame()